# **ML Optimal BUS Allocation System**

## **Data Pre-Processing**

Import the EHS and EPC datasets.

Note:
1. EHS dataset - 2018-2019 Teaching dataset
2. EPC dataset - 2008-2023

In [9]:
import pandas as pd
ehs = pd.read_csv(r"C:\Users\Oke\Documents\UCL MSc ESDA\ESDA_Term 2\BENV0092 Energy Data Analytics in the Built Environment\Final Coursework\datasets\EHS_dataset_2018-2019\UKDA-8717-tab\tab\ehs_1819_teaching.tab",
                  sep='\t')
ehs.head()

,serialanon,aagfh18,aagfh18new,gorehs,hhcomp1,hhtype6,hhsizex,ndepchild,noUnits1,agehrp4x,...,tenure2,tenure4,hhltsick,mortwkx,rentwkx,amthbenx,housbenx,JOINTINCx,HYEARGRx,hhvulx
0,18190000002,810.653613,0.462654,4,7,5,1,0,1,3,...,4,2,2,-9.000000,86.769231,0.0,2,6760.00000,6760.00000,2
1,18190000005,2757.284121,1.573631,10,7,5,1,0,1,3,...,2,1,1,80.769231,-9.000000,-9.0,-9,41175.00000,41175.00000,2
2,18190000006,708.592029,0.404406,1,1,1,2,0,1,4,...,4,2,1,-9.000000,107.000000,0.0,2,13336.00000,13336.00000,2
3,18190000008,3458.741069,1.973965,10,1,2,4,1,1,3,...,2,1,2,570.623527,-9.000000,-9.0,-9,54317.52966,54317.52966,2
4,18190000012,862.639366,0.492324,5,1,1,2,0,1,3,...,1,1,1,0.000000,-9.000000,-9.0,-9,46602.50000,46602.50000,2


In [10]:
ehs.columns

Index(['serialanon', 'aagfh18', 'aagfh18new', 'gorehs', 'hhcomp1', 'hhtype6',
       'hhsizex', 'ndepchild', 'noUnits1', 'agehrp4x', 'agehrp6x', 'emphrpx',
       'ethhrp2x', 'sexhrp', 'bedstdx', 'nbedsx', 'bedrqx', 'nrooms1a',
       'sharer', 'accomhh1', 'lenresb', 'ftbuyer', 'tenure2', 'tenure4',
       'hhltsick', 'mortwkx', 'rentwkx', 'amthbenx', 'housbenx', 'JOINTINCx',
       'HYEARGRx', 'hhvulx'],
      dtype='object')

In [11]:
# The dataset contains a lot of columns, but we are only interested in a few of them.
ehs_cols = [
    'gorehs',          # Government office region (gor) EHS version
    'hhtype6',           # Household type
    'hhsizex',           # Number of persons in the household
    'tenure2',           # Tenure group (or use 'tenure4' if preferred)
    'hhvulx',            # Household vulnerable (means-tested/disability benefits)
    'housbenx',         # Receives housing benefit
    'HYEARGRx'          # Household gross annual income
]

# Select only the columns we are interested in and rename them for clarity
ehs = ehs[ehs_cols]
ehs = ehs.rename(columns={
    'gorehs': 'gov_region',
    'hhtype6': 'household_type',
    'hhsizex': 'household_size',
    'tenure2': 'tenure_group',
    'hhvulx': 'household_vulnerable',
    'housbenx': 'receives_housing_benefit',
    'HYEARGRx': 'household_gross_income'
})

# Display the first few rows of the cleaned dataset
ehs.head()

,gov_region,household_type,household_size,tenure_group,household_vulnerable,receives_housing_benefit,household_gross_income
0,4,5,1,4,2,2,6760.00000
1,10,5,1,2,2,-9,41175.00000
2,1,1,2,4,2,2,13336.00000
3,10,2,4,2,2,-9,54317.52966
4,5,1,2,1,2,-9,46602.50000


In [12]:
epc = pd.read_csv(r"C:\Users\Oke\Documents\UCL MSc ESDA\ESDA_Term 2\BENV0092 Energy Data Analytics in the Built Environment\Final Coursework\datasets\epc_dataset\domestic-E06000001-Hartlepool\certificates.csv",
                  sep=',')
epc.head(2)

C:\Users\Oke\AppData\Local\Temp\ipykernel_21232\2177963814.py:1: DtypeWarning: Columns (34,37) have mixed types. Specify dtype option on import or set low_memory=False.
  epc = pd.read_csv(r"C:\Users\Oke\Documents\UCL MSc ESDA\ESDA_Term 2\BENV0092 Energy Data Analytics in the Built Environment\Final Coursework\datasets\epc_dataset\domestic-E06000001-Hartlepool\certificates.csv",


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE,REPORT_TYPE
0,0003352408dad177e827f77c62ed7172f2f16336c44dcd...,21 HERIOT GRANGE,HARTLEPOOL,NaN,TS25 3JH,10000150044,E,B,53,83,...,Hartlepool,HARTLEPOOL,England and Wales: 1983-1990,2021-08-06 07:35:31,Rented (social),5.0,NaN,1.001108e+11,Energy Assessor,100
1,1514938939222017012914223649268943,"9, Sea View Terrace",NaN,NaN,TS24 0ET,4423379478,D,B,66,82,...,Hartlepool,HARTLEPOOL,England and Wales: 1950-1966,2017-01-29 14:22:36,owner-occupied,NaN,NaN,1.001100e+11,Address Matched,100


In [13]:
epc.columns

Index(['LMK_KEY', 'ADDRESS1', 'ADDRESS2', 'ADDRESS3', 'POSTCODE',
       'BUILDING_REFERENCE_NUMBER', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
       'POTENTIAL_ENERGY_EFFICIENCY', 'PROPERTY_TYPE', 'BUILT_FORM',
       'INSPECTION_DATE', 'LOCAL_AUTHORITY', 'CONSTITUENCY', 'COUNTY',
       'LODGEMENT_DATE', 'TRANSACTION_TYPE', 'ENVIRONMENT_IMPACT_CURRENT',
       'ENVIRONMENT_IMPACT_POTENTIAL', 'ENERGY_CONSUMPTION_CURRENT',
       'ENERGY_CONSUMPTION_POTENTIAL', 'CO2_EMISSIONS_CURRENT',
       'CO2_EMISS_CURR_PER_FLOOR_AREA', 'CO2_EMISSIONS_POTENTIAL',
       'LIGHTING_COST_CURRENT', 'LIGHTING_COST_POTENTIAL',
       'HEATING_COST_CURRENT', 'HEATING_COST_POTENTIAL',
       'HOT_WATER_COST_CURRENT', 'HOT_WATER_COST_POTENTIAL',
       'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF', 'MAINS_GAS_FLAG', 'FLOOR_LEVEL',
       'FLAT_TOP_STOREY', 'FLAT_STOREY_COUNT', 'MAIN_HEATING_CONTROLS',
       'MULTI_GLAZE_PROPORTION', 'GLAZED_TYPE', 'GLAZED_AREA',
      

In [14]:
# Retain only the columns we are interested in
epc_cols = [
    'LMK_KEY',
    'UPRN',
    'POSTCODE',
    'PROPERTY_TYPE',
    'BUILT_FORM',
    'CURRENT_ENERGY_RATING',
    'POTENTIAL_ENERGY_RATING',
    'HEATING_COST_CURRENT',
    'HEATING_COST_POTENTIAL',
    'CO2_EMISSIONS_CURRENT',
    'CO2_EMISSIONS_POTENTIAL',
    'TOTAL_FLOOR_AREA',
    'MAINHEAT_DESCRIPTION',
    'MAIN_FUEL',
    'CONSTRUCTION_AGE_BAND',
    'LOCAL_AUTHORITY',
    'COUNTY'
]

epc = epc[epc_cols]
epc.head(2)

,LMK_KEY,UPRN,POSTCODE,PROPERTY_TYPE,BUILT_FORM,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,HEATING_COST_CURRENT,HEATING_COST_POTENTIAL,CO2_EMISSIONS_CURRENT,CO2_EMISSIONS_POTENTIAL,TOTAL_FLOOR_AREA,MAINHEAT_DESCRIPTION,MAIN_FUEL,CONSTRUCTION_AGE_BAND,LOCAL_AUTHORITY,COUNTY
0,0003352408dad177e827f77c62ed7172f2f16336c44dcd...,1.001108e+11,TS25 3JH,Flat,End-Terrace,E,B,119.0,107.0,3.2,1.6,36.0,Electric storage heaters,electricity (not community),England and Wales: 1983-1990,E06000001,NaN
1,1514938939222017012914223649268943,1.001100e+11,TS24 0ET,House,Mid-Terrace,D,B,666.0,596.0,3.5,1.9,88.0,"Boiler and radiators, mains gas",mains gas (not community),England and Wales: 1950-1966,E06000001,NaN


In [15]:
# Import ONS Postcode Directory data
# This dataset contains information about the local authority and region for each postcode in the UK.
ons_postcode = pd.read_csv(r"C:\Users\Oke\Downloads\ONSPD_FEB_2024_UK\Data\ONSPD_FEB_2024_UK.csv", sep=',')
ons_postcode.head(2)

C:\Users\Oke\AppData\Local\Temp\ipykernel_21232\1455980668.py:3: DtypeWarning: Columns (18,31,40,45,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  ons_postcode = pd.read_csv(r"C:\Users\Oke\Downloads\ONSPD_FEB_2024_UK\Data\ONSPD_FEB_2024_UK.csv", sep=',')


,pcd,pcd2,pcds,dointr,doterm,oscty,ced,oslaua,osward,parish,...,long,lep1,lep2,pfa,imd,calncv,icb,oa21,lsoa21,msoa21
0,AB1 0AA,AB1 0AA,AB1 0AA,198001,199606.0,S99999999,S99999999,S12000033,S13002843,S99999999,...,-2.242851,S99999999,NaN,S23000009,6715,S99999999,S99999999,NaN,NaN,NaN
1,AB1 0AB,AB1 0AB,AB1 0AB,198001,199606.0,S99999999,S99999999,S12000033,S13002843,S99999999,...,-2.246308,S99999999,NaN,S23000009,6715,S99999999,S99999999,NaN,NaN,NaN


In [16]:
ons_postcode.columns

Index(['pcd', 'pcd2', 'pcds', 'dointr', 'doterm', 'oscty', 'ced', 'oslaua',
       'osward', 'parish', 'usertype', 'oseast1m', 'osnrth1m', 'osgrdind',
       'oshlthau', 'nhser', 'ctry', 'rgn', 'streg', 'pcon', 'eer', 'teclec',
       'ttwa', 'pct', 'itl', 'statsward', 'oa01', 'casward', 'npark', 'lsoa01',
       'msoa01', 'ur01ind', 'oac01', 'oa11', 'lsoa11', 'msoa11', 'wz11',
       'sicbl', 'bua11', 'buasd11', 'ru11ind', 'oac11', 'lat', 'long', 'lep1',
       'lep2', 'pfa', 'imd', 'calncv', 'icb', 'oa21', 'lsoa21', 'msoa21'],
      dtype='object')

In [17]:
# Keep only the necessary columns from ons_postcode
ons_region = ons_postcode[['pcd', 'rgn']].copy()

# Standardize postcodes for matching (remove spaces, uppercase)
epc['POSTCODE'] = epc['POSTCODE'].str.replace(' ', '').str.upper()
ons_region['pcd'] = ons_region['pcd'].str.replace(' ', '').str.upper()

In [18]:
# Merge EPC with ONS region mapping
epc = epc.merge(ons_region, how='left', left_on='POSTCODE', right_on='pcd')

# Drop the duplicate pcd column
epc.drop(columns=['pcd'], inplace=True)

In [19]:
# Mapping ONS region code (rgn) to EHS gov_region code (gorehs equivalent)
rgn_to_gorehs = {
    'E12000001': 1,  # North East
    'E12000002': 2,  # North West
    'E12000003': 4,  # Yorkshire and the Humber
    'E12000004': 5,  # East Midlands
    'E12000005': 6,  # West Midlands
    'E12000006': 7,  # East of England
    'E12000007': 8,  # London
    'E12000008': 9,  # South East
    'E12000009': 10  # South West
}

# Add matching EHS-style gov_region code to EPC
epc['gov_region'] = epc['rgn'].map(rgn_to_gorehs)

In [20]:
# Merge on government office region
merged_data = pd.merge(ehs, epc, on='gov_region', how='inner')
# Display the first few rows of the merged dataset
merged_data.head(2)

,gov_region,household_type,household_size,tenure_group,household_vulnerable,receives_housing_benefit,household_gross_income,LMK_KEY,UPRN,POSTCODE,...,HEATING_COST_POTENTIAL,CO2_EMISSIONS_CURRENT,CO2_EMISSIONS_POTENTIAL,TOTAL_FLOOR_AREA,MAINHEAT_DESCRIPTION,MAIN_FUEL,CONSTRUCTION_AGE_BAND,LOCAL_AUTHORITY,COUNTY,rgn
0,1,1,2,4,2,2,13336.0,0003352408dad177e827f77c62ed7172f2f16336c44dcd...,1.001108e+11,TS253JH,...,107.0,3.2,1.6,36.0,Electric storage heaters,electricity (not community),England and Wales: 1983-1990,E06000001,NaN,E12000001
1,1,1,2,4,2,2,13336.0,1514938939222017012914223649268943,1.001100e+11,TS240ET,...,596.0,3.5,1.9,88.0,"Boiler and radiators, mains gas",mains gas (not community),England and Wales: 1950-1966,E06000001,NaN,E12000001


In [21]:
merged_data.columns

Index(['gov_region', 'household_type', 'household_size', 'tenure_group',
       'household_vulnerable', 'receives_housing_benefit',
       'household_gross_income', 'LMK_KEY', 'UPRN', 'POSTCODE',
       'PROPERTY_TYPE', 'BUILT_FORM', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'HEATING_COST_CURRENT',
       'HEATING_COST_POTENTIAL', 'CO2_EMISSIONS_CURRENT',
       'CO2_EMISSIONS_POTENTIAL', 'TOTAL_FLOOR_AREA', 'MAINHEAT_DESCRIPTION',
       'MAIN_FUEL', 'CONSTRUCTION_AGE_BAND', 'LOCAL_AUTHORITY', 'COUNTY',
       'rgn'],
      dtype='object')

In [22]:
# Keep only the relevant columns for the final dataset
selected_features = [
    'gov_region',
    'household_type',
    'household_size',
    'tenure_group',
    'household_vulnerable',
    'receives_housing_benefit',
    'household_gross_income',
    'PROPERTY_TYPE',
    'BUILT_FORM',
    'CURRENT_ENERGY_RATING',
    'HEATING_COST_CURRENT',
    'CO2_EMISSIONS_CURRENT',
    'TOTAL_FLOOR_AREA',
    'MAIN_FUEL',
    'CONSTRUCTION_AGE_BAND'
]

final_data = merged_data[selected_features]
# Display the first few rows of the final dataset
final_data.head(2)

,gov_region,household_type,household_size,tenure_group,household_vulnerable,receives_housing_benefit,household_gross_income,PROPERTY_TYPE,BUILT_FORM,CURRENT_ENERGY_RATING,HEATING_COST_CURRENT,CO2_EMISSIONS_CURRENT,TOTAL_FLOOR_AREA,MAIN_FUEL,CONSTRUCTION_AGE_BAND
0,1,1,2,4,2,2,13336.0,Flat,End-Terrace,E,119.0,3.2,36.0,electricity (not community),England and Wales: 1983-1990
1,1,1,2,4,2,2,13336.0,House,Mid-Terrace,D,666.0,3.5,88.0,mains gas (not community),England and Wales: 1950-1966
